In [ ]:
import subprocess
from pydub import AudioSegment
import math
import os
import time


def check_ffmpeg():
    try:
        result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True)
        if result.returncode == 0:
            print("ffmpeg is installed and working.")
            print(result.stdout)
        else:
            print("ffmpeg is installed but not working correctly.")
            print(result.stderr)
            return False
    except FileNotFoundError:
        print("ffmpeg is not installed or not found in PATH.")
        return False
    return True


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-y",  # Add this option to automatically overwrite existing files
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    try:
        with subprocess.Popen(
            command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        ) as process:
            while True:
                output = process.stderr.read(1)
                if output == "" and process.poll() is not None:
                    break
                if output:
                    print(output, end="")
        if process.returncode != 0:
            print("ffmpeg process failed")
    except subprocess.TimeoutExpired:
        print("ffmpeg process timed out")
    except Exception as e:
        print(f"Error during ffmpeg execution: {e}")


if check_ffmpeg():
    # Extract audio from video
    video_path = "./files/jocoding-gpt.mp4"
    audio_path = "./files/jocoding-gpt.mp3"
    start_time = time.time()
    print("Starting audio extraction")
    extract_audio_from_video(video_path, audio_path)
    extraction_time = time.time() - start_time
    print(f"Audio extraction time: {extraction_time} seconds")

    # Check if audio file was created
    if not os.path.exists(audio_path):
        print(f"Audio extraction failed, {audio_path} not found")
    else:
        try:
            # Load audio file
            start_time = time.time()
            print("Loading audio file")
            track = AudioSegment.from_mp3(audio_path)
            loading_time = time.time() - start_time
            print(f"Audio loading time: {loading_time} seconds")

            # Define chunk length (10 minutes)
            ten_minutes = 10 * 60 * 1000

            # Calculate number of chunks
            chunks = math.ceil(len(track) / ten_minutes)
            print(f"Total number of chunks: {chunks}")

            # Ensure chunks directory exists
            chunks_dir = "./files/chunks"
            if not os.path.exists(chunks_dir):
                os.makedirs(chunks_dir)

            # Export each chunk as a separate file
            for i in range(chunks):
                start_time = time.time()
                start_time_ms = i * ten_minutes
                end_time_ms = (i + 1) * ten_minutes
                chunk = track[start_time_ms:end_time_ms]
                chunk_filename = f"jocoding-gpt_{i + 1:02d}.mp3"
                chunk_path = os.path.join(chunks_dir, chunk_filename)
                chunk.export(chunk_path, format="mp3")
                print(f"Chunk {i + 1} export time: {time.time() - start_time} seconds")
        except Exception as e:
            print(f"Error during audio processing: {e}")
else:
    print("Please install ffmpeg or ensure it is in the PATH")